# **Student Performance Indicator**

<h1 style="font-family: 'poppins'; font-weight: bold; color: Orange;">Author: Ali Nawaz</h1>

[![GitHub](https://img.shields.io/badge/GitHub-Profile-green?style=for-the-badge&logo=github)](https://github.com/Aliktk) [![LinkedIn](https://img.shields.io/badge/LinkedIn-Profile-blue?style=for-the-badge&logo=linkedin)](https://www.linkedin.com/in/ali-nawaz-khattak/)


---
### **Experimentation for End to end Generative AI Project: MCQ Generator using OpenAI, Langchain Streamlit**
---


#### **Importing Required Libraries**


In [1]:
import pandas as pd
import PyPDF2
import json
import traceback
import os
from dotenv import load_dotenv

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback

#### **Setting up OPENAI API KEY**


In [2]:
load_dotenv()  # load environment variables from .env file

openai_api_key = os.getenv("OPENAI_API_KEY")  # get api key from environment variable

#### **Create an instance of LLM with api key and model name**


In [4]:
llm = ChatOpenAI(openai_api_key=openai_api_key, model_name="gpt-4", temperature=0.5)

C:\Users\Nawaz\AppData\Local\Temp\ipykernel_17532\3637529922.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(openai_api_key=openai_api_key, model_name="gpt-4",temperature=0.5)


The provided code snippet initializes an instance of the ChatOpenAI class from the openai library, configured with an OpenAI API key, the "gpt-4" model, and a temperature parameter of 0.5.


#### **Define Prompt Template**


In [5]:
# sample response json for prompt template to generate quiz

RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

#### **Prompt template to generate quiz from text**


In [6]:
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [7]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE,
)

#### **Define llm chain to generate quiz**


In [8]:
quiz_chain = LLMChain(
    llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True
)

C:\Users\Nawaz\AppData\Local\Temp\ipykernel_17532\2520680470.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  quiz_chain = LLMChain(


#### **Prompt template2 to generate quiz from text**


In [9]:
# prompt template2 to generate quiz from text

TEMPLATE2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

#### **This prompt template is used to evaluate the complexity of the quiz**


In [43]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject", "quiz"], template=TEMPLATE
)

#### **Review chain to evaluate the quiz and give feedback**


In [44]:
review_chain = LLMChain(
    llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True
)

#### **Sequential chain to generate quiz and evaluate it**


In [45]:
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["text", "number", "subject", "tone", "response_json"],

    output_variables=["quiz", "review"],
    verbose=True,
)

#### **Defining Path of file that we will use to generate mcqs**


In [46]:
file_path = r"data/MCQ_Training_Data.txt"

#### **Read text from file_path**


In [47]:
with open(file_path, "r") as file:
    TEXT = file.read()

print(TEXT)

#### **Serialize the Python dictionary into a JSON-formatted string**


In [48]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

#### **This code will define tone of the quiz**


In [49]:
NUMBER = 5
SUBJECT = "Machine Learning"
TONE = "Professional"  # You can choose between "simple", "formal", "professional"

#### **How to setup Token Usage Tracking in LangChain**


In [50]:
# https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON),
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:
You are an expert MCQ maker. Given the above text, it is your job to create a quiz  of 5 multiple choice questions for Machine Learning students in Professional tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make 5 MCQs
### RESPONSE_JSON
{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}




#### **Cost of above operation**


In [55]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1165
Prompt Tokens:502
Completion Tokens:663
Total Cost:0.05483999999999999


#### **Response**


In [56]:
print(response)

{'text': '', 'number': 5, 'subject': 'Machine Learning', 'tone': 'Professional', 'response_json': '{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}', 'quiz': '{"1": {"mcq": "What does MCQ stand for in the context of education?", "options": {"a": "Machine Code Query", "b": "Multiple Choice Question", "c": "Multiple Code Quotient", "d": "Machine Choice Questionnaire"}, "correct": "b"}, "2": {"mcq": "What is the role of an expert MCQ maker?", "options": {"a": "To create quizzes", "b": "To design websites", "c": "To analyze data", "d": "To manage project

#### **Response in JSON**


In [57]:
quiz_str = response.get("quiz")

#### **Use json.loads on the quiz_str to convert it from a JSON string to a dictionary.**


In [58]:
quiz = json.loads(quiz_str)

#### **This code is used to convert the quiz to a table**


In [59]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [55]:
quiz_table_data

[{'MCQ': 'What is the main goal of machine learning?',
  'Choices': 'a: To perform tasks with explicit instructions | b: To learn from data and generalize to unseen data | c: To ignore data and focus on theoretical frameworks | d: To memorize data without understanding it',
  'Correct': 'b'},
 {'MCQ': "Who coined the term 'machine learning' in 1959?",
  'Choices': 'a: Donald Hebb | b: Walter Pitts | c: Arthur Samuel | d: Warren McCulloch',
  'Correct': 'c'},
 {'MCQ': 'What is the difference between machine learning and data mining?',
  'Choices': 'a: Machine learning focuses on prediction, while data mining focuses on discovering unknown properties in data | b: Machine learning uses unsupervised learning, while data mining uses supervised learning | c: Machine learning is used for image processing, while data mining is used for speech recognition | d: Machine learning is a subset of data mining',
  'Correct': 'a'},
 {'MCQ': 'What is the goal of generalization in machine learning?',
  '

#### **Make a dataframe of the quiz**


In [60]:
pd.DataFrame(quiz_table_data)

,MCQ,Choices,Correct
0,What does MCQ stand for in the context of educ...,a: Machine Code Query | b: Multiple Choice Que...,b
1,What is the role of an expert MCQ maker?,a: To create quizzes | b: To design websites |...,a
2,What is the target audience for the quiz menti...,a: High school students | b: Machine Learning ...,b
3,How many multiple choice questions does the te...,a: 10 | b: 5 | c: 20 | d: 15,b
4,What should be ensured about the questions to ...,a: They should be repeated | b: They should be...,c


In [61]:
quiz = pd.DataFrame(quiz_table_data)

#### **Save it in csv file**


In [65]:
quiz.to_csv("data/Machine_Learning_Quiz.csv", index=False)

In [64]:
from datetime import datetime

datetime.now().strftime("%m_%d_%Y_%H_%M_%S")

'09_16_2024_00_49_16'